In [2]:
%pip install earthengine-api -q
%pip install folium -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import ee

In [4]:
ee.Authenticate()


Successfully saved authorization token.


In [5]:
ee.Initialize()

In [6]:
collection = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CO').select('CO_column_number_density').filterDate('2019-06-01', '2019-06-11')

In [7]:
import folium

def add_ee_layer(self, ee_image_object, vis_params, name, opacity = 0.5):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True,
        opacity=opacity,
    ).add_to(self)
    
folium.Map.add_ee_layer = add_ee_layer

In [8]:
import pandas as pd

def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df

In [38]:
from IPython.display import display

def display_map(position, image: ee.Image, band_viz_min=0, band_viz_max=0.05):
    band_viz = {
      'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
    }
    if band_viz_max: band_viz["max"] = band_viz_max
    if band_viz_min: band_viz["min"] = band_viz_min
    my_map = folium.Map(location=position, zoom_start=10)
    my_map.add_ee_layer(image, band_viz, 'S5P CO', opacity=0.4)
    display(my_map)

In [10]:
collection = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CO').select('CO_column_number_density').filterDate('2022-01-01', '2022-02-22')
collection_median = collection.median()

In [26]:
kernel = ee.Kernel.circle(10)
collection_convoluted = collection_median.convolve(kernel)

In [36]:
collection_impact = collection_median.subtract(collection_convoluted)

In [27]:
position = (1.7, -74.8)

In [28]:
display_map(position, collection_median)

In [33]:
display_map(position, collection_convoluted)

In [45]:
display_map(position, collection_impact)